In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv(r"D:\covid_prediction\data\covid.csv")

In [3]:
x=df.drop("Has_Covid",axis=1)
y=df["Has_Covid"]

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

In [5]:
X_train_processed=X_train.copy()
X_test_processed=X_test.copy()

In [6]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])

In [7]:
X_train_processed['Cough']=oe.fit_transform(X_train[['Cough']])
X_test_processed['Cough']=oe.transform(X_test[['Cough']])

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
le_gender=LabelEncoder()
X_train_processed['Gender']=le_gender.fit_transform(X_train[['Gender']])
X_test_processed['Gender']=le_gender.transform(X_test[['Gender']])

C:\Users\Asus\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Asus\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [10]:
le_target=LabelEncoder()
y_train_encoded=le_target.fit_transform(y_train)
y_test_encoded=le_target.transform(y_test)

In [11]:
# all_cities=pd.get_dummies(X_train['City'],prefix='City').columns
all_cities = df['City'].unique()

In [12]:
X_train_city=pd.get_dummies(X_train['City'],prefix='City')
X_test_city=pd.get_dummies(X_test['City'],prefix='City')

In [13]:
# X_train_city=X_train_city.reindex(columns=all_cities,fill_value=0)
# X_test_city=X_test_city.reindex(columns=all_cities,fill_value=0)
X_train_city = X_train_city.reindex(columns=[f'City_{c}' for c in all_cities], fill_value=0)
X_test_city = X_test_city.reindex(columns=[f'City_{c}' for c in all_cities], fill_value=0)

In [14]:
X_train_processed=X_train_processed.drop('City',axis=1)
X_test_processed=X_test_processed.drop('City',axis=1)

In [15]:
X_train_processed=pd.concat([X_train_processed,X_train_city],axis=1)
X_train_processed=pd.concat([X_test_processed,X_test_city],axis=1)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [17]:
num_cols=['Age','Fever']

In [18]:
X_train_processed[num_cols]=scaler.fit_transform(X_train_processed[num_cols])
X_test_processed[num_cols]=scaler.transform(X_test_processed[num_cols])

In [19]:
X_train_processed = X_train_processed.astype(float)
X_test_processed = X_test_processed.astype(float)

In [20]:
X_train_scaled = X_train_processed
X_test_scaled = X_test_processed

In [21]:
import joblib

joblib.dump(oe,r"D:\covid_prediction\models\cough_encoder.pkl")
joblib.dump(le_gender,r"D:\covid_prediction\models\gender_encoder.pkl")
joblib.dump(le_target,r"D:\covid_prediction\models\target_encoder.pkl")
joblib.dump(scaler,r"D:\covid_prediction\models\scaler.pkl")
joblib.dump(all_cities,r"D:\covid_prediction\models\city_column.pkl")
joblib.dump(list(X_train_scaled.columns), r"D:\covid_prediction\models\feature_names.pkl")

['D:\\covid_prediction\\models\\feature_names.pkl']

In [22]:
X_train_processed.dtypes

Age               float64
Gender            float64
Fever             float64
Cough             float64
City_Mumbai       float64
City_Kolkata      float64
City_Bangalore    float64
City_Delhi        float64
dtype: object